# Imports

In [2]:
import transformers
import torch
import pandas as pd
import numpy as np
import re

#transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BertTokenizer, BertModel
from bert_score import BERTScorer
from torchmetrics.text import CHRFScore
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu

#utilities
import xml.etree.ElementTree as ET
import itertools
import codecs
from tqdm import tqdm


# Dataset

In [3]:
# RU

datapath = "/data/danielstorage/ru-with-refs.xml"

root = ET.parse(datapath)
ru_predicates = []

for e in root.iterfind('./entries/entry'):
    category = e.get('category')
    eid = e.get('eid')

    triple_list = []
    entity = [category, eid]
    triples = []
    for mtriple in e.findall('./modifiedtripleset/mtriple'):

        trans = {}
        for dbpedialink in e.findall('./dbpedialinks/dbpedialink'):
          txt = dbpedialink.text
          parts = txt.split('|')
          trans[parts[0].strip()] = parts[2].strip()

        for link in e.findall('./links/link'):
          txt = link.text
          parts = txt.split('|')
          part0 = parts[0].strip().replace(' ', '_')
          trans[part0] = parts[2].strip()

        txt = mtriple.text
        triple_list.append(txt)
        unstripped_parts = txt.split('|')
        parts = [part.strip() for part in unstripped_parts]
        assert len(parts)==3

        translated_parts = []
        for part in parts:
          if part in trans:
            translated_parts.append(trans[part])
          else:
            translated_parts.append(part)

        triples.append(translated_parts)

    entity.append(triples)

    lex = e.findall('./lex')
    lex = [l.text for l in lex]
    entity.append(lex)
    ru_predicates.append(entity)

# EN

datapath = "/data/danielstorage/en-with-refs.xml"

root = ET.parse(datapath)
en_predicates = []

for e in root.iterfind('./entries/entry'):
    category = e.get('category')
    eid = e.get('eid')

    entity = [category, eid]
    triples = []
    for mtriple in e.findall('./modifiedtripleset/mtriple'):
      txt = mtriple.text
      unstripped_parts = txt.split('|')
      parts = [part.strip() for part in unstripped_parts]
      assert len(parts) == 3

      triples.append(parts)

    entity.append(triples)

    lex = e.findall('./lex')
    lex = [l.text for l in lex]
    entity.append(lex)
    en_predicates.append(entity)

# CREATING THE CORRECT INPUT FORMAT
# EN
input_texts_en = []

for predicate in en_predicates:
  subject = predicate[0]
  id = predicate[1]
  originaltext = predicate[3]
  input_entity = {"subject": subject, "id": id, "originaltext": originaltext}
  input_text = ""
  for sentence in predicate[2]:
    # Format: "subject has relation_to object"
    input_text += f"{sentence[0]} has {sentence[1]} {sentence[2]}. "
  cleaned_input_text = re.sub(r'[\\\"\']', '', input_text)
  cleaned_input_text = cleaned_input_text.replace('_', ' ')
  input_entity["inputtext"] = cleaned_input_text
  input_texts_en.append(input_entity)

# RU
input_texts_ru = []

for predicate in ru_predicates:
  subject = predicate[0]
  id = predicate[1]
  originaltext = predicate[3]
  input_entity = {"subject": subject, "id": id, "originaltext": originaltext}
  input_text = ""
  for sentence in predicate[2]:
    # Format: "subject has relation_to object"
    input_text += f"{sentence[0]} имеет {sentence[1]} {sentence[2]}. "
  cleaned_input_text = re.sub(r'[\\\"\']', '', input_text)
  cleaned_input_text = cleaned_input_text.replace('_', ' ')
  input_entity["inputtext"] = cleaned_input_text
  input_texts_ru.append(input_entity)



In [5]:
en_predicates[0]

['SportsTeam',
 'Id1',
 [['Estádio_Municipal_Coaracy_da_Mata_Fonseca', 'location', 'Arapiraca'],
  ['Agremiação_Sportiva_Arapiraquense',
   'league',
   'Campeonato_Brasileiro_Série_C'],
  ['Campeonato_Brasileiro_Série_C', 'country', 'Brazil'],
  ['Agremiação_Sportiva_Arapiraquense', 'nickname', '"\'\'Alvinegro"'],
  ['Agremiação_Sportiva_Arapiraquense',
   'ground',
   'Estádio_Municipal_Coaracy_da_Mata_Fonseca']],
 ['Estádio Municipal Coaracy da Mata Fonseca is the name of the ground of Agremiação Sportiva Arapiraquense in Arapiraca. Agremiação Sportiva Arapiraquense, nicknamed "Alvinegro", lay in the Campeonato Brasileiro Série C league from Brazil.',
  'Estádio Municipal Coaracy da Mata Fonseca is the name of the ground of Agremiação Sportiva Arapiraquense in Arapiraca. Alvinegro, the nickname of Agremiação Sportiva Arapiraquense, play in the Campeonato Brasileiro Série C league from Brazil.']]

In [7]:
ru_predicates[0]

['Food',
 'Id1',
 [['“Запеченная Аляска”', 'course', 'Десерт'],
  ['Десерт', 'dishVariation', 'Печенье'],
  ['Соединённые_Штаты_Америки', 'ethnicGroup', 'Афроамериканцы'],
  ['“Запеченная Аляска”', 'country', 'Соединённые_Штаты_Америки']],
 ['“Запеченная Аляска” и печенье – десерты. "Запеченная Аляска" происходит из Соединённых Штатов Америки, в которых этническая группа - Афроамериканцы.',
  '"Запеченная Аляска" и печенье - разновидность десерта и происходит из Соединённых Штатов Америки. Афроамериканцы - одна из этнических групп этой страны.']]

In [6]:
print(f"Length: {len(input_texts_en)}")
input_texts_en[0]

Length: 1779


{'subject': 'SportsTeam',
 'id': 'Id1',
 'originaltext': ['Estádio Municipal Coaracy da Mata Fonseca is the name of the ground of Agremiação Sportiva Arapiraquense in Arapiraca. Agremiação Sportiva Arapiraquense, nicknamed "Alvinegro", lay in the Campeonato Brasileiro Série C league from Brazil.',
  'Estádio Municipal Coaracy da Mata Fonseca is the name of the ground of Agremiação Sportiva Arapiraquense in Arapiraca. Alvinegro, the nickname of Agremiação Sportiva Arapiraquense, play in the Campeonato Brasileiro Série C league from Brazil.'],
 'inputtext': 'Estádio Municipal Coaracy da Mata Fonseca has location Arapiraca. Agremiação Sportiva Arapiraquense has league Campeonato Brasileiro Série C. Campeonato Brasileiro Série C has country Brazil. Agremiação Sportiva Arapiraquense has nickname Alvinegro. Agremiação Sportiva Arapiraquense has ground Estádio Municipal Coaracy da Mata Fonseca. '}

In [6]:
print(f"Length: {len(input_texts_ru)}")
input_texts_ru[0]

Length: 1102


{'subject': 'Food',
 'id': 'Id1',
 'originaltext': ['“Запеченная Аляска” и печенье – десерты. "Запеченная Аляска" происходит из Соединённых Штатов Америки, в которых этническая группа - Афроамериканцы.',
  '"Запеченная Аляска" и печенье - разновидность десерта и происходит из Соединённых Штатов Америки. Афроамериканцы - одна из этнических групп этой страны.'],
 'inputtext': '“Запеченная Аляска” имеет course Десерт. Десерт имеет dishVariation Печенье. Соединённые Штаты Америки имеет ethnicGroup Афроамериканцы. “Запеченная Аляска” имеет country Соединённые Штаты Америки. '}

# Models

- Phi4
- LLama3
- Gemma

In [5]:
# Phi4
phi4_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-4-mini-instruct",
    device_map = "cuda",
    torch_dtype = "auto",
    trust_remote_code = True,
)
phi4_tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-4-mini-instruct")

# Create a pipeline
phi4_pipeline = pipeline(
    "text-generation",
    model = phi4_model,
    tokenizer = phi4_tokenizer,
    return_full_text=False,
    trust_remote_code=False,
    max_new_tokens = 200,
    do_sample = False
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:23<00:00, 11.74s/it]
Device set to use cuda


In [4]:
# LLama3

llama3_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.1-8B-Instruct",
    device_map = "cuda",
    torch_dtype = "auto",
    trust_remote_code = True,
)
llama3_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct", return_token_type_ids = False)
llama3_tokenizer.chat_template = "<|begin_of_text|>{% if not add_generation_prompt is defined %}{% set add_generation_prompt = false %}{% endif %}{% for message in messages %}{{'<|start_header_id|>' + message['role'] + '<|end_header_id|>' + '\n\n' + message['content'] + '<|eot_id|>'}}{% endfor %}{% if add_generation_prompt %}{{ '<|start_header_id|>assistant\n' }}{% endif %}"

llama3_pipeline = pipeline(
    "text-generation",
    model = llama3_model,
    tokenizer = llama3_tokenizer,
    return_full_text = False,
    model_kwargs = {"torch_dtype": torch.bfloat16},
    device_map = "auto",
    max_new_tokens = 200 
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:52<00:00, 13.04s/it]
Device set to use cuda


In [4]:
# Gemma
gemma_model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-3-4b-it",
    device_map = "cuda",
    torch_dtype = "auto",
    trust_remote_code = True,
)
gemma_tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-4b-it", return_token_type_ids = False)

gemma_tokenizer.chat_template = "<bos>{% if not add_generation_prompt is defined %}{% set add_generation_prompt = false %}{% endif %}{% for message in messages %}{{'<start_of_turn>' + message['role'] + '\n' + message['content'] + '<end_of_turn>'}}{% endfor %}{% if add_generation_prompt %}{{ '<bos>assistant\n' }}{% endif %}"


# Create a pipeline
gemma_pipeline = pipeline(
    "text-generation",
    model = gemma_model,
    tokenizer = gemma_tokenizer,
    return_full_text=False,
    trust_remote_code=False,
    max_new_tokens = 200,
    do_sample = False
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:28<00:00, 14.50s/it]
Device set to use cuda


# Generation
- Run the code block under here first; it sets the model you will be using!!

In [6]:
# Phi4

generator = phi4_pipeline
modelname = "Phi4"

In [5]:
# Llama3

generator = llama3_pipeline
modelname = "Llama3"

In [5]:
# Gemma

generator = gemma_pipeline
modelname = "Gemma"

In [6]:
# Setting up functions and dataframes for metrics & results

ru_results = pd.DataFrame(columns = ["Language", "Model", "BeamSearch", "Temperature", "Category", "Input String", "Original", "Generated", "TextID", "BERTScore F1", "chrF++", "Sentence BLEU", "Corpus BLEU"])
en_results = pd.DataFrame(columns = ["Language", "Model", "BeamSearch", "Temperature", "Category", "Input String", "Original", "Generated", "TextID", "BERTScore F1", "chrF++", "Sentence BLEU", "Corpus BLEU"])

# Scorers for metrics
bertscorer = BERTScorer(model_type='bert-base-uncased')
chrfscorer = CHRFScore()


In [20]:
# RU

beamsearch_beams = np.nan    # NaN or 4
temperature = 1.4   # NaN or 0.8 / 1.4 # MAKE SURE TO SWITCH INPUTS IN generator() !!!!

ru_hypotheses_list = []
ru_originaltext_list = []

for input_entity in tqdm(input_texts_ru):
    subject = input_entity["subject"]
    id = input_entity["id"]
    inputtext = input_entity["inputtext"]
    originaltext = input_entity["originaltext"]
    
    messages = [ 
        {"role": "assistant", "content": "Вам надо преобразовать следующие тройки в естественный текст. Предоставляйте только текст, соответствующий тройкам, не возвращайте вводный текст. Предоставляйте на русском языке."},
        {"role": "user", "content": inputtext}
    ]
    
    # Generate output
    output = generator(
        messages,
        do_sample = True,

        # In case of beamsearch: 
        #num_beams = beamsearch_beams,
        #early_stopping = True,

        # In case of temperature:
        temperature = temperature
    )
    generatedtext = output[0]["generated_text"]

    ru_hypotheses_list.append(generatedtext)
    ru_originaltext_list.append(originaltext)
    print(generatedtext, '\n')
    
    # Calculating metrics
    
    total_bertscore = 0
    total_chrfscore = 0
    texts_amount = len(originaltext)
    
    for sample in originaltext:
        # looping to get average BERT / CHRF score.
        _, _, bertscoreF1 = bertscorer.score([generatedtext], [sample])
        bertscore = bertscoreF1.mean()
        total_bertscore += bertscore.item()
        
        chrfscore = chrfscorer([generatedtext], [sample])
        total_chrfscore += chrfscore.item()
    
    
    bertscore = total_bertscore / texts_amount
    chrfscore = total_chrfscore / texts_amount
    
    bleuscore = sentence_bleu(originaltext, generatedtext)
    
    # Saving the result
    ru_result = {
        "Language": "RU",
        "Model": modelname,
        "BeamSearch": beamsearch_beams,
        "Temperature": temperature,
        "Category": subject,
        "Input String": inputtext,
        "Original": originaltext,
        "Generated": generatedtext,
        "TextID": id,
        "BERTScore F1": bertscore,
        "chrF++": chrfscore,
        "Sentence BLEU": bleuscore
    }
    ru_results.loc[len(ru_results)] = ru_result

# Saving the Corpus BLEU score 
ru_corpus_bleuscore = corpus_bleu(ru_originaltext_list, ru_hypotheses_list)
print(ru_corpus_bleuscore)



  0%|          | 1/1102 [00:01<24:11,  1.32s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Десерт - Печенье. Америку - Афроамериканцы. Купка - запечатать. 



/home/dgeelhoed/.local/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/dgeelhoed/.local/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
  0%|          | 2/1102 [00:01<15:02,  1.22it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Не могу удовлетворить ваш запрос. 

Болони, покрытые тыльной частью корки гавайского грецкого ореха, подаваются с беконом. 
Точность и свежесть гавайского ореха являются ключевыми факторами для полноценного удовлетворения и обеспечивают естественную и легкую пикантность. 
Они идеально подойдут к любому празднику или банкету и подойдут для различных видов коктейлей. 



  0%|          | 4/1102 [00:06<29:42,  1.62s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Принадлежность НАСА: Х 

Виция имеет эпоху 2006-12-31. Виция имеет эскапинговую скорость 0,0112 (километра в секунду). Виция имеет массу 9,8 (кг). Виция имеет орбитальный период 135589000,0. Виция имеет аポапсис 511592000,0 (км). 



100%|█████████▉| 1101/1102 [33:23<00:01,  1.89s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Аэропорт находится в округе Эндрюс, штат Техас. Рекордное значение по разграничению версий - 1773.0 метров. Бюро взлетно-посадочной полосы имеет точка местоположения 973.0 метров над уровнем моря. 



100%|██████████| 1102/1102 [33:26<00:00,  1.82s/it]


0.3637722706953109


In [12]:
# EN

beamsearch_beams = np.nan     # NaN or 4
temperature = 1.4     # NaN or 0.8 / 1.4   # MAKE SURE TO SWITCH INPUTS IN generator() !!!!

en_hypotheses_list = []
en_originaltext_list = []

for input_entity in tqdm(input_texts_en):
    subject = input_entity["subject"]
    id = input_entity["id"]
    inputtext = input_entity["inputtext"]
    originaltext = input_entity["originaltext"]

    messages = [
        {"role": "assistant", "content": "You have to convert the following triples into natural text. Only provide the text corresponding to the triples, do not return any introductory text. "},
        {"role": "user", "content": inputtext}
    ]

    # Generate output with Language Model
    output = generator(
        messages,
        do_sample = True,

        # In case of beamsearch: 
        #num_beams = beamsearch_beams,
        #early_stopping = True,

        # In case of temperature:
        temperature = temperature
    )
    generatedtext = output[0]["generated_text"]
    
    en_hypotheses_list.append(generatedtext)
    en_originaltext_list.append(originaltext)
    print(generatedtext, '\n')
    
    

    # Calculating metrics

    total_bertscore = 0
    total_chrfscore = 0
    texts_amount = len(originaltext)
    
    for sample in originaltext:
    # looping to get average BERT / CHRF score.
        _, _, bertscoreF1 = bertscorer.score([generatedtext], [sample])
        bertscore = bertscoreF1.mean()
        total_bertscore += bertscore.item()
        
        chrfscore = chrfscorer([generatedtext], [sample])
        total_chrfscore += chrfscore.item()


    bertscore = total_bertscore / texts_amount
    chrfscore = total_chrfscore / texts_amount
    
    bleuscore = sentence_bleu(originaltext, generatedtext)
    
    # Saving the result
    en_result = {
        "Language": "EN",
        "Model": modelname,
        "BeamSearch": beamsearch_beams,
        "Temperature": temperature,
        "Category": subject,
        "Input String": inputtext,
        "Original": originaltext,
        "Generated": generatedtext,
        "TextID": id,
        "BERTScore F1": bertscore,
        "chrF++": chrfscore,
        "Sentence BLEU": bleuscore
    }
    en_results.loc[len(en_results)] = en_result

en_corpus_bleuscore = corpus_bleu(en_originaltext_list, en_hypotheses_list)
print(en_corpus_bleuscore)



  0%|          | 1/1779 [00:02<1:25:49,  2.90s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Arapiraca is located at Estádio Municipal Coaracy da Mata Fonseca. Alvinegro competes in Campeonato Brasileiro Série C. Campeonato Brasileiro Série C is a Brazilian football league. Alvinegro plays at Estádio Municipal Coaracy da Mata Fonseca. 



  0%|          | 2/1779 [00:03<51:04,  1.72s/it]  Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Nie Haisheng, born on October 13, 1964, is a fighter pilot. 



  0%|          | 3/1779 [00:04<32:58,  1.11s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

100%|█████████▉| 1778/1779 [51:07<00:01,  1.53s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The album Turn Me On is a Punk blues album produced by The Honeymoon Killers. 



100%|██████████| 1779/1779 [51:08<00:00,  1.72s/it]

Ciudad Ayala is located in Mexico. Its standard timezone is Pacific Time Zone. During daylight saving time, its timezone shifts to Pacific Daylight Time. 



0.6480610107764853


# Evaluation

In [24]:
ru_results.at[2204, 'Corpus BLEU'] =  ru_corpus_bleuscore

In [13]:
en_results.at[1779, 'Corpus BLEU'] =  en_corpus_bleuscore

In [8]:
len(en_results)


1779

In [15]:
en_results.loc[1779]

Language                                                        EN
Model                                                       Llama3
BeamSearch                                                     NaN
Temperature                                                    1.4
Category                                                SportsTeam
Input String     Estádio Municipal Coaracy da Mata Fonseca has ...
Original         [Estádio Municipal Coaracy da Mata Fonseca is ...
Generated        Arapiraca is located at Estádio Municipal Coar...
TextID                                                         Id1
BERTScore F1                                              0.687276
chrF++                                                    0.439115
Sentence BLEU                                             0.548191
Corpus BLEU                                               0.648061
Name: 1779, dtype: object

In [19]:
en_results.loc[1779]

Language                                                        EN
Model                                                         Phi4
BeamSearch                                                     NaN
Temperature                                                    0.8
Category                                                SportsTeam
Input String     Estádio Municipal Coaracy da Mata Fonseca has ...
Original         [Estádio Municipal Coaracy da Mata Fonseca is ...
Generated        Arapiraca is located in the Estádio Municipal ...
TextID                                                         Id1
BERTScore F1                                              0.783005
chrF++                                                    0.676035
Sentence BLEU                                             0.640679
Corpus BLEU                                                    NaN
Name: 1779, dtype: object

In [17]:
rownr = 9
for col in ru_results.columns:
    print(f"{col}:")
    print(ru_results.loc[rownr][col])
    print()
    

Language:
RU

Model:
Phi4

Category:
Building

Input String:
Дом Ашера и Мэри Изабель Ричардсон имеет added to the National Register of Historic Places 1988-11-22. 

Original:
['Дом Ашера и Мэри Изабель Ричардсон был добавлен в Национальный реестр исторических мест 22 ноября 1988 года.']

Generated:
Дом Ашера и Мэри Изабель Ричардсон был добавлен в Национальный реестр исторических мест 22 ноября 1988 года.

TextID:
Id10

BERTScore F1:
1.0

chrF++:
1.0

BLEU:
1.0



In [15]:
rownr = 1
for col in en_results.columns:
    print(f"{col}:")
    print(en_results.loc[rownr][col])
    print()

Language:
EN

Model:
Phi4

Category:
Astronaut

Input String:
Nie Haisheng has birthDate 1964-10-13. Nie Haisheng has occupation Fighter pilot. 

Original:
['Nie Haisheng, born on October 13, 1964, worked as a fighter pilot.', 'Nie Haisheng is a former fighter pilot who was born on October 13, 1964.', 'Nie Haisheng born on 10/13/1964 is a fighter pilot.']

Generated:
Nie Haisheng was born on October 13, 1964, and is a fighter pilot.

TextID:
Id2

BERTScore F1:
0.8561983307202657

chrF++:
0.7130934397379557

BLEU:
0.9286099819098308



In [26]:
ru_results.to_csv('ru_results.csv')

In [16]:
en_results.to_csv('en_results.csv')